<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/container_at_adv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.8/323.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.6/512.6 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [ ]:
# worker reservation
worker_name = "worker-01"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# advantech router reservation
adv_name = "adv-02"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-02-26 20:13:16.900 | INFO     | chi.expeca:reserve:167 - reserving adv-02
2024-02-26 20:13:19.342 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for adv-02-lease with id b56a828e-5b35-4908-abc6-cfd0fe057e64 to become "ACTIVE"
2024-02-26 20:13:24.539 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-02-lease with id b56a828e-5b35-4908-abc6-cfd0fe057e64 is PENDING.
2024-02-26 20:13:29.738 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-02-lease with id b56a828e-5b35-4908-abc6-cfd0fe057e64 is PENDING.
2024-02-26 20:13:34.927 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-02-lease with id b56a828e-5b35-4908-abc6-cfd0fe057e64 is PENDING.
2024-02-26 20:13:40.132 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-02-lease with id b56a828e-5b35-4908-abc6-cfd0fe057e64 is PENDING.
2024-02-26 20:13:45.336 | INFO     | chi.expeca:wait_until_lease_status:69 - lease adv-02-lease with id b56a828e-5b35-4908-abc6-cfd

[
    {
        "name": "worker-01-lease",
        "id": "0461c496-d6bf-4fb3-a0b0-9ea2c1027be7",
        "reservation_id": "e650b561-3ab2-496a-95ce-e1e0db25ea6d",
        "status": "ACTIVE",
        "end_date": "2024-03-02T13:18:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "1336affd-6aef-45f4-aa53-5870914b9c95",
        "reservation_id": "728feadf-98ba-4c7d-b7a1-6e6116488191",
        "status": "ACTIVE",
        "end_date": "2024-03-04T19:30:00.000000"
    },
    {
        "name": "adv-02-lease",
        "id": "b56a828e-5b35-4908-abc6-cfd0fe057e64",
        "reservation_id": "1b8cbc56-384a-44de-8973-524db4c2968d",
        "status": "ACTIVE",
        "end_date": "2024-03-04T20:13:00.000000"
    }
]


In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info). It contains creation of an edge-net, a router, and some interfaces on the router and routes.

NOTE: You must choose a network interface on the worker only with 1Gbps speed not 10Gbps.

In [ ]:
# check available 1Gbps interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    if interfaces[interface]['speed'] == '1000':
      available_ifs.append(interface)
logger.info(f"Available 1Gbps interfaces on {worker_name}: {available_ifs}")
if len(available_ifs) < 1:
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find enough 1Gbps interfaces on {worker_name}")

# run the container
advnet = chi.network.get_network(adv_name+"-net")
container_name = adv_name+"-end-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2024-02-26 20:20:00.377 | SUCCESS  | __main__:<cell line: 18>:18 - created samie-end-node container.


In [ ]:
command = "irtt client --tripm=oneway -i 10ms -f 5ms -g edaf_ep5g/adv02 -l 500 -m 1 -d 5m -o d --outdir=/tmp/ 10.70.70.3 > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref=container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2024-02-26 20:25:15.472 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
